# 0. Initialize

## 0.1. Import Libraries

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os, sys, glob
import gzip
import random
import tqdm
import json
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)

from IPython import display
import matplotlib as mpl
from matplotlib import pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 0.2. DEFINE VARIABLES 

In [ ]:
DATA_PATH = '/Users/onurvarol/Downloads/data/' # '<insert-your-training-data-path-here>'

ROUND = 1 # This project will have 3 rounds of predictions: 1,2,3
STUDENT_ID = 'baseline'#'<insert-your-id-here>'
PROJECT_CODE = 'baseline'#'<insert-your-code-here>' # Same code for the annotation eg. CS412xxxxx

## 0.3. Read Training & Evaluation Data

### 0.3.1. Get the labels for tweets

In [ ]:
#trainingTweetDf = pd.read_csv('{}training-tweet.csv'.format(DATA_PATH))
trainingTweetDf = pd.read_csv('{}training-tweet.csv'.format(DATA_PATH), dtype={'tweet_id': str, 'isPolitical': str})
trainingTweetDf

tweet_id isPolitical
0     1597170281545551872         Yes
1     1431700027471192069          No
2     1566035577090281472         Yes
3     1591538690869940225         Yes
4     1583898169238167554         Yes
...                   ...         ...
2995  1593539327623151619         Yes
2996  1393886554062524418          No
2997  1597925615092764672         Yes
2998  1585291418616176640         Yes
2999  1594775287622995971         Yes

[3000 rows x 2 columns]

In [ ]:
trainingTweetDf.isPolitical.value_counts()

Yes    2003
No      997
Name: isPolitical, dtype: int64

### 0.3.2. Get the labels for users

In [ ]:
trainingUserDf = pd.read_csv('{}training-user.csv'.format(DATA_PATH))
#trainingUserDf = pd.read_csv('training-user.csv')
trainingUserDf

screen_name isBot
0       koftecancaddy    No
1              ahaber    No
2     selahat03949652    No
3       erdin06357062    No
4       bhct__necatii    No
...               ...   ...
2995     djblumenberg    No
2996           mel1sq    No
2997         eren_yz1   Yes
2998      ergnyildiz4    No
2999   isadal24978794   Yes

[3000 rows x 2 columns]

In [ ]:
trainingUserDf.isBot.value_counts()

No     2424
Yes     576
Name: isBot, dtype: int64

### 0.3.3. Expand your dataset with metadata and tweets

In [ ]:
# You can also expand training data by downloading your own labeled datasets following the link
# Download the documents under "Link to training data"

print('http://www.onurvarol.com/Annotation-CS412-202201/reports/report_{}.html'.format(PROJECT_CODE))

http://www.onurvarol.com/Annotation-CS412-202201/reports/report_baseline.html


# 1. EXTRACT FEATURES
Under *1.1. Political Tweet Detection* and *1.2. Bot Detection*, we firstly collect raw data for processing. We then combine some of them (total_interactions = num_favorites + num_retweets) or use them to extract features (whether the tweet has one of the political entities @meralaksener, @kilicdarogluk etc.).

We expect you to collect more raw data from **tweet_metadata**, **user_profiles** and **user_tweets** files by creating a function as shown in below examples such as *check_if_retweet()* and using it while iterating over data as shown under *Merge Collected Features*.

We also expect you to create new variables as much as you can from the data in order to make your predictions more accurate. For example, you may want to check:

- The tweet sources that a user frequently uses
- Whether the user is a verified account or not

...

to assess whether **a user is a bot or not** and whether **a tweet is political or not**.

In [ ]:
PATH_TO_DOWNLOADED = DATA_PATH # 'D:/Users/suuser/Desktop/Sabancı/CS412/spring-2022/project/'

## 1.1. Political Tweet Detection
This part stands for the feature extraction of tweets. We start with collecting the raw data from *tweet_metadata*, then use some of them to extract features.

### 1.1.1. Get Raw Data

#### 1.1.1.1. Check if Retweet

In [ ]:
def check_if_retweet(tweet_metadata_line):
    is_retweet = 0
    retweeted_username = None

    try:
        tweet_metadata_line['retweeted_status']
        retweeted_username = tweet_metadata_line['retweeted_status']['user']['screen_name'].lower()
        is_retweet = 1

    except KeyError:
        pass

    return is_retweet, retweeted_username

#### 1.1.1.2. Get Tweet Text

In [ ]:
def get_tweet_text(tweet_metadata_line):
    text = tweet_metadata_line['text']
    
    return text

#### 1.1.1.3. Get Tweet ID

In [ ]:
def get_tweet_id(tweet_metadata_line):
    id_str = tweet_metadata_line['id_str']
    
    return id_str

#### 1.1.1.4. Get Number of Mentions and Hashtags

In [ ]:
def get_number_mentions_hashtags(tweet_metadata_line):
    num_mentions = len(tweet_metadata_line['entities']['user_mentions'])
    num_hashtags = len(tweet_metadata_line['entities']['hashtags'])

    return num_mentions, num_hashtags

#### 1.1.1.5. Get Number of Retweets and Favorites

In [ ]:
def get_number_retweets_favorites(tweet_metadata_line):
    retweet_count = tweet_metadata_line['retweet_count']
    favorite_count = tweet_metadata_line['favorite_count']
    
    return retweet_count, favorite_count

#### 1.1.1.6. Get User Info

In [ ]:
def get_user_info(tweet_metadata_line):
    id = tweet_metadata_line['user']['id_str']
    screen_name = tweet_metadata_line['user']['screen_name'].lower()
    description = tweet_metadata_line['user']['description']

    return id, screen_name, description

### 1.1.2. Derive Manually Crafted Features

#### 1.1.2.1. Check for political entity in text

In [ ]:
def check_political_ent(text):
    
    # the list below can be modified and some new names may be added (or removed)
    list_of_entities = ['meral_aksener', 'kilicdarogluk', 'vekilince', 'RTErdogan', 'MevlutCavusoglu', 'umitozdag']
    
    entities_in_text = [ent for ent in list_of_entities if ent.lower() in text.lower()]
    number_entities = len(entities_in_text)

    return number_entities

#### 1.1.2.2. Number of total interactions

In [ ]:
def total_interactions(retweet_count, favorite_count):
    total_num_interactions = retweet_count + favorite_count
    
    return total_num_interactions

### 1.1.2. Collect data using the functions above and transform into a Pandas DataFrame

In [ ]:
dfPolitical = {'tweet_id':[],
              'is_retweet':[],
              'retweeted_username':[],
              'text':[],
              'num_mentions':[],
              'num_hashtags':[],
              'num_retweets':[],
              'num_favorites':[],
              'user_id':[],
              'user_screen_name':[],
              'user_description':[],
              'num_political_entities':[],
              'total_interactions':[]}


with gzip.open(f"{PATH_TO_DOWNLOADED}tweet_metadata.jsons.gz", "rb") as f:
    for line in f:
        line = json.loads(line)
        
        # raw data:
        id_str = get_tweet_id(line)
        is_retweet, retweeted_username = check_if_retweet(line)
        text = get_tweet_text(line)
        num_mentions, num_hashtags = get_number_mentions_hashtags(line)
        retweet_count, favorite_count = get_number_retweets_favorites(line)
        user_id_str, screen_name, user_description = get_user_info(line)

        # manually crafted data:
        num_political_entities = check_political_ent(text)
        total_num_interactions = total_interactions(retweet_count, favorite_count)

        dfPolitical['tweet_id'].append(id_str)
        dfPolitical['is_retweet'].append(is_retweet)
        dfPolitical['retweeted_username'].append(retweeted_username)
        dfPolitical['text'].append(text)
        dfPolitical['num_mentions'].append(num_mentions)
        dfPolitical['num_hashtags'].append(num_hashtags)
        dfPolitical['num_retweets'].append(retweet_count)
        dfPolitical['num_favorites'].append(favorite_count)
        dfPolitical['user_id'].append(user_id_str)
        dfPolitical['user_screen_name'].append(screen_name)
        dfPolitical['user_description'].append(user_description)
        dfPolitical['num_political_entities'].append(num_political_entities)
        dfPolitical['total_interactions'].append(total_num_interactions)

In [ ]:
dfPolitical = pd.DataFrame(dfPolitical)
dfPolitical

tweet_id  is_retweet retweeted_username  \
0      1588568792984346624           0               None   
1      1588452263047069697           0               None   
2      1569589330544398336           0               None   
3      1570428119609139201           0               None   
4      1551163840368414722           0               None   
...                    ...         ...                ...   
33528  1568595408233832448           0               None   
33529  1584027427696959488           0               None   
33530  1585945783307730945           0               None   
33531  1569748909521801221           1       muazzezeralp   
33532  1585926877675282432           0               None   

                                                    text  num_mentions  \
0      Sosyal Hizmetin temelini çocuk oluşturur,çocuğ...             0   
1      @mahirunal Gavur İzmir ya onlar, hani Cumhuriy...             1   
2      #ŞehitAdayıUzmÇvşaKadro\nSiz İstesenizde Istem...             0   
3      @ajans_muhbir Siz kaypak olmayıp onay vermesey...             1   
4      Engelli öğretmenler olarak önümüzdeki engeller...             0   
...                                                  ...           ...   
33528  Gerçek kimlik taşımayan hesaplara cevap vermem...             0   
33529  @umitozdag Neden Suriyelilerle ilgili bu kadar...             1   
33530  @celebimehmeta Niye Türkiye yüzyılıda.Türkiye ...             1   
33531  RT @muazzezeralp: @Doan58213655 @denizkonur @N...             7   
33532  @bybekirbozdag Yasal işlemlerdn dolayı( banka,...             1   

       num_hashtags  num_retweets  num_favorites              user_id  \
0                 0            49             98   920963718103650304   
1                 0             0              0            595514060   
2                 1             0              0  1356375754561490947   
3                 0             0              0  1478775431008595968   
4                 0             0              0  1511976696337113088   
...             ...           ...            ...                  ...   
33528             0             9             81            576247173   
33529             0             1              8            162308585   
33530             0             0              1            415025519   
33531             1             6              0  1442125177727307781   
33532             0             0              0  1180170443426320384   

      user_screen_name                                   user_description  \
0             maviruh_     shu/\nburaya afilli bir söz yazdığımı varsayın   
1               mtfdan                                                      
2      ahsucilginuzman                                    Vatan Sevdalisi   
3         hamitelkelle                                            HighOne   
4          sed58417690                                                      
...                ...                                                ...   
33528     ardanzenturk  RT ONAYLADIĞIM ANLAMINA GELMEZ\nArtık fikirler...   
33529         ozgul_61      Bridge design engineer Yaay hesabı : dilfiruz   
33530        ladrekova                                                      
33531  yapikytgrivrlsn                                                      
33532  rodrige62822813  Canı yanan sabretsin; can yakan da yanacağı gü...   

       num_political_entities  total_interactions  
0                           0                 147  
1                           0                   0  
2                           0                   0  
3                           0                   0  
4                           0                   0  
...                       ...                 ...  
33528                       0                  90  
33529                       1                   9  
33530                       0                   1  
33531                       1                   6  
33532                 

## 1.2. From Users

### 1.2.1. Get user metadata from user_profiles.jsons.gz

#### 1.2.1.1. Get user info metadata

In [ ]:
def get_user_info_metadata(user_metadata_line):
    
    user_id = user_metadata_line['id_str']
    user_name = user_metadata_line['name']
    user_screen_name = user_metadata_line['screen_name'].lower()
    user_location = user_metadata_line['location']
    user_description = user_metadata_line['description']
    user_followers_count = user_metadata_line['followers_count']
    user_friends_count = user_metadata_line['friends_count']
    
    dictionary = {'user_id':user_id, 'user_name': user_name, 'user_screen_name':user_screen_name, 'user_location':user_location,
     'user_description':user_description, 'user_followers_count':user_followers_count, 'user_friends_count':user_friends_count}

    return dictionary

#### 1.2.1.2. Get followers/(followers+friends) ratio

In [ ]:
def get_followers_all_ratio(user_followers_count, user_friends_count):
    
    if user_friends_count + user_followers_count == 0:
        followers_all_ratio = 0

    else:
        followers_all_ratio =  user_followers_count / (user_friends_count + user_followers_count)

    return followers_all_ratio

#### 1.2.1.3. Get description length

In [ ]:
def get_desc_len(user_description):
    
    description_len = len(user_description)

    return description_len

In [ ]:
dfBot = {'user_id':[],
         'user_name':[],
         'user_screen_name':[],
         'user_location':[],
         'user_description':[],
         'user_followers_count':[],
         'user_friends_count':[],
         'description_len':[],
         'followers_to_all_ratio':[]}

with gzip.open(f"{PATH_TO_DOWNLOADED}user_profiles.jsons.gz", "rb") as f:
    for line in f:
        line = json.loads(line)

        dictionary = get_user_info_metadata(line)
        for k,v in dictionary.items():
            dfBot[k].append(v)

        
        # manually crafted data:
        description_len = get_desc_len(dictionary['user_description'])
        dfBot['description_len'].append(description_len)
        
        followers_all_ratio = get_followers_all_ratio(dictionary['user_followers_count'], 
                                                      dictionary['user_friends_count'])

        dfBot['followers_to_all_ratio'].append(followers_all_ratio)

In [ ]:
dfBot = pd.DataFrame(dfBot)
dfBot

user_id                                      user_name  \
0      1431241870848450577                            Nasreena Khan Wazir   
1      1304340303080386560                                    fania :((((   
2      1116042038577958914                                    Yusuf Aksoy   
3               4859899931  Be (VIXX6) ama oppalarının düğününe gidemiyor   
4               2225373636                                         SLMDMR   
...                    ...                                            ...   
29665  1320834618220781569                                        islammm   
29666            111074128                                     Melda Onur   
29667             36946875                                        ali ydm   
29668           2389587396                                    Türkan Usta   
29669           1575904134                                           ARAZ   

      user_screen_name            user_location  \
0      nasreenakhan006      Islamabad, Pakistan   
1          scorpiehoez                   bogor    
2      yusufak63712920                            
3       nedenburdaysam  Hufflepuff ortak salon    
4          biologselim                            
...                ...                      ...   
29665       islam_mrsj                            
29666        meldaonur                            
29667          ali_ydm        İstanbul, Türkiye   
29668       turkanusta          Ankara, Türkiye   
29669        ben121975                            

                                        user_description  \
0                                                Student   
1                                    have a holly jolly🎄   
2      Bir şeyden pişmanlık duymak istemiyorsan,her ş...   
3      SMStan\n/St☆rlight ///come on girls,this is ou...   
4                               BİYOLOG🔬🦠\nNanoteknoloji   
...                                                  ...   
29665                                                      
29666  Şekersiz çay, etsiz sofra, SAVAŞSIZ dünya... 🐌...   
29667                       hayat oyunsa bende jeton çok   
29668  Ustaya sormuşlar; hayatta yaptığın en büyük is...   
29669  Korkak gerçek le yüzleşmeyi redd eder,hırçınla...   

       user_followers_count  user_friends_count  description_len  \
0                        65                 185                7   
1                      8235                3011               19   
2                        95                 399               64   
3                        40                  83               65   
4                       100                  98               23   
...                     ...                 ...              ...   
29665                   229                 217                0   
29666                212457                5428              100   
29667                   116                 141               28   
29668                  1669                3639              116   
29669                  6026                5625              158   

       followers_to_all_ratio  
0                    0.260000  
1                    0.732260  
2                    0.192308  
3                    0.325203  
4                    0.505051  
...                       ...  
29665                0.513453  
29666                0.975088  
29667                0.451362  
29668                0.314431  
29669                0.517209  

[29670 rows x 9 columns]

### 1.2.2. Get Tweet Info of Users in user_profiles.jsons.gz

#### 1.2.2.1. Check ratio of retweets to all tweets

In [ ]:
def get_retweet_tweet_ratio(line):
    number_retweets = 0
    number_original_tweets = 0

    for tweet in line['tweets']:
        try:
            tweet['retweeted_status']
            number_retweets += 1
                
        except:
            number_original_tweets += 1
            
    total_tweets = number_retweets + number_original_tweets
    
    if total_tweets == 0:
        retweet_total_ratio = None
    else:
        retweet_total_ratio = number_retweets/(total_tweets)
    
    return retweet_total_ratio

#### 1.2.2.2. Check median number of favorites

In [ ]:
def get_median_number_favorites(line):
    num_median_favorites = np.median([tweet['favorite_count'] for tweet in line['tweets']])

    return num_median_favorites

### 1.2.3. Collect data using the functions above and transform into a Pandas DataFrame

In [ ]:
dfBotTweets = {'user_id':[],
               'retweet_total_ratio':[],
               'num_median_favorites':[],
               'num_of_tweets':[]
              }

i = 0

with gzip.open(f"{PATH_TO_DOWNLOADED}user_tweets.jsons.gz", "rb") as f:
    for line in f:

        line = json.loads(line)

        user_id = line['user_id']
        dfBotTweets['user_id'].append(user_id)
        
        retweet_total_ratio = get_retweet_tweet_ratio(line)
        dfBotTweets['retweet_total_ratio'].append(retweet_total_ratio)
        
        num_median_favorites = get_median_number_favorites(line)
        dfBotTweets['num_median_favorites'].append(num_median_favorites)
        
        dfBotTweets['num_of_tweets'].append(len(line['tweets']))

        i += 1
        if i % 1000 == 0:
            print(i)

/Users/onurvarol/anaconda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/onurvarol/anaconda/lib/python3.6/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000


In [ ]:
dfBotTweets = pd.DataFrame(dfBotTweets)
dfBotTweets

user_id  retweet_total_ratio  num_median_favorites  \
0                594642154             0.115000                   2.0   
1                525600289             0.005025                   1.0   
2       931895965501534209             0.900000                   0.0   
3      1591543462746329088             0.185000                   0.0   
4       734801354749796352             1.000000                   0.0   
...                    ...                  ...                   ...   
28310  1591370361488252928             0.800000                   0.0   
28311  1475272459616235525             0.825000                   0.0   
28312  1096753792731750401             0.051020                   1.0   
28313  1269527617687953409             0.095000                   2.0   
28314  1483177252271570944             0.630000                   0.0   

       num_of_tweets  
0                200  
1                199  
2                200  
3                200  
4                200  
...              ...  
28310            200  
28311            200  
28312            196  
28313            200  
28314            200  

[28315 rows x 4 columns]

### 1.2.3. Merge dfBot and dfBotTweets

In [ ]:
dfBotAll = dfBot.merge(dfBotTweets,
                       how='left')

dfBotAll[['retweet_total_ratio', 'num_median_favorites']] = dfBotAll[['retweet_total_ratio', 'num_median_favorites']].fillna(0)

dfBotAll

user_id                                      user_name  \
0      1431241870848450577                            Nasreena Khan Wazir   
1      1304340303080386560                                    fania :((((   
2      1116042038577958914                                    Yusuf Aksoy   
3               4859899931  Be (VIXX6) ama oppalarının düğününe gidemiyor   
4               2225373636                                         SLMDMR   
...                    ...                                            ...   
29665  1320834618220781569                                        islammm   
29666            111074128                                     Melda Onur   
29667             36946875                                        ali ydm   
29668           2389587396                                    Türkan Usta   
29669           1575904134                                           ARAZ   

      user_screen_name            user_location  \
0      nasreenakhan006      Islamabad, Pakistan   
1          scorpiehoez                   bogor    
2      yusufak63712920                            
3       nedenburdaysam  Hufflepuff ortak salon    
4          biologselim                            
...                ...                      ...   
29665       islam_mrsj                            
29666        meldaonur                            
29667          ali_ydm        İstanbul, Türkiye   
29668       turkanusta          Ankara, Türkiye   
29669        ben121975                            

                                        user_description  \
0                                                Student   
1                                    have a holly jolly🎄   
2      Bir şeyden pişmanlık duymak istemiyorsan,her ş...   
3      SMStan\n/St☆rlight ///come on girls,this is ou...   
4                               BİYOLOG🔬🦠\nNanoteknoloji   
...                                                  ...   
29665                                                      
29666  Şekersiz çay, etsiz sofra, SAVAŞSIZ dünya... 🐌...   
29667                       hayat oyunsa bende jeton çok   
29668  Ustaya sormuşlar; hayatta yaptığın en büyük is...   
29669  Korkak gerçek le yüzleşmeyi redd eder,hırçınla...   

       user_followers_count  user_friends_count  description_len  \
0                        65                 185                7   
1                      8235                3011               19   
2                        95                 399               64   
3                        40                  83               65   
4                       100                  98               23   
...                     ...                 ...              ...   
29665                   229                 217                0   
29666                212457                5428              100   
29667                   116                 141               28   
29668                  1669                3639              116   
29669                  6026                5625              158   

       followers_to_all_ratio  retweet_total_ratio  num_median_favorites  \
0                    0.260000             0.395939                   0.0   
1                    0.732260             0.125000                   0.0   
2                    0.192308             0.910000                   0.0   
3                    0.325203             0.015306                   1.0   
4                    0.505051             0.659898                   0.0   
...                       ...                  ...                   ...   
29665                0.513453             0.015000                   1.0   
29666                0.975088             0.291457                   2.0   
29667                0.451362             0.061538                   0.0   
29668                0.314431             0.995000                   0.0   
29669                0.517209             0.030000                   1.0   

       num_of_tweets  
0      

# 2. TRAIN MODEL

## 2.1. Political Tweet Prediction

### 2.1.1. Merge dfPolitical data with labels

In [ ]:
dfPoliticalAll_train = dfPolitical.merge(trainingTweetDf,
                                         on='tweet_id')

dfPoliticalAll_train.head()

tweet_id  is_retweet retweeted_username  \
0  1585955683513798656           0               None   
1  1597631718479261696           0               None   
2  1572522789948751874           0               None   
3  1591412481561624577           0               None   
4  1596914274907348992           0               None   

                                                text  num_mentions  \
0  @AvOzlemZengin YüzüncüYıla YakışanGenelAf adli...             1   
1  #TCYüzyılıÜcretliÖgrtKadro\n#TCYüzyılıÜcretliÖ...             0   
2  Ekrem İmamoğlu davayı değerlendirdi. 'Boş işle...             0   
3  Sayın Bakanım @suleymansoylu POMEM önlisans er...             1   
4  @varank Sayın bakanım,  Bodrumdaki bu araziyi ...             1   

   num_hashtags  num_retweets  num_favorites              user_id  \
0             0             3              2  1564992353168941058   
1             2            30             28  1324630334416297985   
2             0             5             66            407597071   
3             0             0              0  1394789887073738753   
4             0             0              0  1586083256088371201   

  user_screen_name                                   user_description  \
0    zehra78231638                                                      
1       nurozguler                                                      
2        onediocom  Türkiye'nin ilk ve tek sosyal içerik sitesi ht...   
3    buckybarnestr                                                ...   
4      sayariahmet                                                      

   num_political_entities  total_interactions isPolitical  
0                       0                   5         Yes  
1                       0                  58         Yes  
2                       0                  71         Yes  
3                       0                   0         Yes  
4                       0                   0         Yes

### 2.1.2. Separate X and y values
We only use 3 features here to create a baseline model. However, it is not enough to get good results.

In [ ]:
X = dfPoliticalAll_train[['num_political_entities','total_interactions','num_hashtags']]
y = dfPoliticalAll_train['isPolitical'].apply(lambda x: 1 if x=='Yes' else 0)

### 2.1.3. Train - validation split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)

### 2.1.4. Train the model

Here, you may use different models such as neural networks, XGBoost, AdaBoost, RandomForest, Linear Regression, Logistic Regression etc. to see which model does the best. Also, you can use grid_search_cv() or a basic for loop to optimize the hyperparameters of your model.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

# create an instance
dtc_political = DecisionTreeClassifier()

# fit your model
dtc_political.fit(X_train, y_train)

# make predictions
preds = dtc_political.predict(X_valid)

# evaluate on validation set
acc_score = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)
mse = mean_squared_error(y_valid, preds)

print("MSE:", mse, "\n",
      "Accuracy Score:", acc_score, "\n",
      "Confusion Matrix:", "\n", confusion)

MSE: 0.35 
 Accuracy Score: 0.65 
 Confusion Matrix: 
 [[ 52 157]
 [ 53 338]]


## 2.2. Bot Detection

### 2.2.1. Merge dfBotAll data with labels

In [ ]:
dfBotAll.user_screen_name = dfBotAll.user_screen_name.str.lower()

In [ ]:
dfBotAll_train = dfBotAll.merge(trainingUserDf,
                               left_on='user_screen_name',
                               right_on='screen_name')

dfBotAll_train

user_id                 user_name user_screen_name  \
0     1512081815292432394                    sezgin  sezgin953116371   
1     1425452291428077571                  Adem Koç       gogoadem61   
2               328164303         Necmettin Balıkçı         dewil511   
3     1343666971368431622              Night Bird⁷🦉   midnight__bird   
4     1240932880488038400             Samed Pınarcı     samedpinarci   
...                   ...                       ...              ...   
2995  1286770207134973954            Hamide Arabacı         anka6054   
2996  1598032338323214338      atamabekleyenbahceci  atamabekleyenzz   
2997   760235343966863360                Emrah İNCİ       memrahinci   
2998  1553973684100124672                 Murat Kkk       muratkkk18   
2999  1392390985645281283  Münferit Fenerbahçeliler  mnferitfenerba1   

           user_location                                   user_description  \
0                                                                             
1                                                                             
2                                                                             
3                          La vie est un sommeil, l’amour en est le rêve...   
4                         Orman Mühendisi - Orman İşletme Şefi - Orman G...   
...                  ...                                                ...   
2995                                                                          
2996                                                                          
2997  Istanbul - Bayburt  Researcher | Middle East | Political Science |...   
2998                                             Normal sıradan bir insanım   
2999                      Fenerbahçe S.K. Kongre Üyesi Kırmızı çizgileri...   

      user_followers_count  user_friends_count  description_len  \
0                       46                 430                0   
1                       14                 171                0   
2                       21                  49                0   
3                      422                 260               48   
4                      133                 202               60   
...                    ...                 ...              ...   
2995                   151                  61                0   
2996                   173                 367                0   
2997                  5863                5905               71   
2998                     1                  10               26   
2999                   281                 270              156   

      followers_to_all_ratio  retweet_total_ratio  num_median_favorites  \
0                   0.096639             0.050251                   0.0   
1                   0.075676             0.761062                   0.0   
2                   0.300000             0.010101                   0.0   
3                   0.618768             0.085000                   1.0   
4                   0.397015             0.780000                   0.0   
...                      ...                  ...                   ...   
2995                0.712264             0.000000                   1.0   
2996                0.320370             0.580000                   0.0   
2997                0.498215             0.040000                  36.0   
2998                0.090909             0.769231                   0.0   
2999                0.509982             0.045000                   1.0   

      num_of_tweets      screen_name isBot  
0             199.0  sezgin953116371    No  
1             113.0       gogoadem61    No  
2             198.0         dewil511   Yes  
3             200.0   midnight__bird    No  
4             200.0     samedpinarci    No  
...             ...              ...   ...  
2995          200.0         anka6054    No  
2996          200.0  atamabekleyenzz    No  
2997          200.0       memrahinci    No  
2998           13.0       muratkkk18    No  
299

In [ ]:
trainingUserDf.isBot.value_counts()

No     2424
Yes     576
Name: isBot, dtype: int64

### 2.2.2. Separate X and y values
We use only 4 features here to create a baseline model. However, it is not enough to get good results.

In [ ]:
X = dfBotAll_train[['description_len', 'followers_to_all_ratio', 'retweet_total_ratio', 'num_median_favorites']]
y = dfBotAll_train.isBot.apply(lambda x: 1 if x=='Yes' else 0)

### 2.2.3. Train-test split

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.20, random_state=42)

### 2.2.4. Train the model

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error

# create an instance
dtc_bot = DecisionTreeClassifier()

# fit your model
dtc_bot.fit(X_train, y_train)

# make predictions
preds = dtc_bot.predict(X_valid)

# evaluate on validation set
acc_score = accuracy_score(y_valid, preds)
confusion = confusion_matrix(y_valid, preds)

print("MSE:", mse, "\n",
      "Accuracy Score:", acc_score, "\n",
      "Confusion Matrix:", "\n", confusion)

MSE: 0.35 
 Accuracy Score: 0.73 
 Confusion Matrix: 
 [[398  95]
 [ 67  40]]


# 3. MAKE PREDICTIONS

Here, you will make predictions with the models that you have trained above.

## 3.1. Predictions for Tweets (Political or Not)

In [ ]:
# read the evaluation file as follows
evaluationTweetDf = pd.read_csv('evaluation-round{}-tweet.csv'.format(ROUND), dtype={0: str}, header=None, names=['tweet_id'])
evaluationTweetDf = evaluationTweetDf.dropna()
evaluationTweetDf

# merge it with the political dataframe so that you can use the make predictions based on the variables
dfPolitical_test = dfPolitical.merge(evaluationTweetDf)

# define X as we did above in section (2.x.2. Separate X and y values)
X = dfPolitical_test[['num_political_entities','total_interactions','num_hashtags']]

# make predictions based on these variables
predictions_political = dtc_political.predict(X)

### This part is important! We expect you to return your predictions in the following format:

In [ ]:
modelPredTweet = dict([(x,float(y)) for x,y in zip([*dfPolitical_test.tweet_id], predictions_political)])
modelPredTweet

{'1593649159009099777': 1.0,
 '1367571642604544000': 1.0,
 '1589993032975544320': 1.0,
 '1565312596135354373': 1.0,
 '1388235183653011462': 1.0,
 '1592120408073203712': 0.0,
 '1439547067337256967': 1.0,
 '1597274845381029888': 1.0,
 '1586021183958704128': 1.0,
 '1356926480605982728': 0.0,
 '1595357036925026306': 1.0,
 '1585766233491886081': 1.0,
 '1595871258985615361': 1.0,
 '1352635736537882629': 1.0,
 '1583477966373543936': 1.0,
 '1564926450096013313': 1.0,
 '1585634359612420101': 1.0,
 '1384499047390658560': 0.0,
 '1596583748669419521': 1.0,
 '1391681495622995971': 0.0,
 '1365710259549966339': 1.0,
 '1590673118397624323': 1.0,
 '1389951943343316995': 0.0,
 '1407921226656280580': 1.0,
 '1452348722810138646': 1.0,
 '1597256187325878273': 1.0,
 '1595829502021623812': 1.0,
 '1579408398894137344': 1.0,
 '1570758749606019073': 1.0,
 '1366091745772077058': 0.0,
 '1596233602886701057': 1.0,
 '1584922292127256577': 1.0,
 '1586279180983042050': 0.0,
 '1399687111234756612': 1.0,
 '136322656468

## 3.2. Predictions for Users (Bot or Not)

In [ ]:
evaluationUserDf = pd.read_csv('evaluation-round{}-user.csv'.format(ROUND), dtype={0: str}, header=None, names=['user_screen_name'])
evaluationUserDf = evaluationUserDf.dropna()

# merge it with the political dataframe so that you can use the make predictions based on the variables
dfBot_test = dfBotAll.merge(evaluationUserDf)

# define X as we did above in section (2.x.2. Separate X and y values)
X = dfBot_test[['description_len', 'followers_to_all_ratio', 'retweet_total_ratio', 'num_median_favorites']]

# make predictions based on these variables
predictions_bot = dtc_bot.predict(X)

In [ ]:
modelPredUser = dict([(x,float(y)) for x,y in zip([*dfBot_test.user_screen_name], predictions_bot)])
modelPredUser

{'nedenburdaysam': 0.0,
 'biologselim': 1.0,
 'alaraaynncnm': 0.0,
 '_sydneycarton_': 0.0,
 'denizlihabercom': 0.0,
 'burakerbaychp': 0.0,
 'mustafaarst': 0.0,
 'mvnez': 0.0,
 'qara118': 0.0,
 'alpar_kaan': 0.0,
 'farukhalit2': 0.0,
 'haf_zhan': 1.0,
 'harlunoshi': 0.0,
 'heritagepaix': 1.0,
 '37baho37': 0.0,
 'tamerduran_1': 0.0,
 'donkisotumsu': 0.0,
 'nuranwolf': 0.0,
 'politikgundem': 1.0,
 'isakethudax': 0.0,
 'ilaydejaneiro': 0.0,
 'gendenmukatol': 0.0,
 '1905anason': 1.0,
 'eraydurgut03': 1.0,
 'dasiskein': 0.0,
 'mett_1907': 0.0,
 'semihyeteer': 0.0,
 'haberinyokcokk': 0.0,
 'meleky_ozaydin': 0.0,
 'han34nesli': 0.0,
 'bilobi4': 0.0,
 'berkeduranovic': 0.0,
 'cagdasadim': 0.0,
 'merabalare': 0.0,
 'sevdaac72373936': 0.0,
 '21gramlife1': 0.0,
 'cakan0_': 0.0,
 'oguzksalici': 0.0,
 'emre_caliskann': 0.0,
 'mehmet07454846': 0.0,
 'lifegs': 1.0,
 'begumkarabeyx': 1.0,
 'avutulan': 0.0,
 'imamgibiimam': 1.0,
 'durdane52': 0.0,
 'radyotrafik35': 0.0,
 'rk_ozanyali': 1.0,
 'uykusuz75'

# PREPARE SUBMISSION

You will need to submit exact same file produced by using the following code. Any deviation from the desired format willbe marked as 0.

In [ ]:
# Explain your approach

data_explanations = '''
Explain here how you handle data for training
'''

feature_explanations = '''
What types of features you created
'''

model_explanations = '''
What did you try and used for modeling
'''

additional_explanations = '''
Any other tricks that you tried for the project
'''


In [ ]:
predictions = {
    'round': ROUND,
    'student_id': STUDENT_ID,
    'user_predictions': modelPredUser,
    'tweet_predictions': modelPredTweet,
    'explanations': {
        'data': data_explanations,
        'feature': feature_explanations,
        'model': model_explanations,
        'other': additional_explanations,
    }
}


with open('predictions-{}_round{}.json'.format(STUDENT_ID, ROUND), 'w') as fl:
    fl.write(json.dumps(predictions, indent=4))

In [ ]:
# Test your submission file

submission = json.load(open('predictions-{}_round{}.json'.format(STUDENT_ID, ROUND), 'r'))
submission

{'explanations': {'data': '\nExplain here how you handle data for training\n',
  'feature': '\nWhat types of features you created\n',
  'model': '\nWhat did you try and used for modeling\n',
  'other': '\nAny other tricks that you tried for the project\n'},
 'round': 1,
 'student_id': 'baseline',
 'tweet_predictions': {'1593649159009099777': 1.0,
  '1367571642604544000': 1.0,
  '1589993032975544320': 1.0,
  '1565312596135354373': 1.0,
  '1388235183653011462': 1.0,
  '1592120408073203712': 0.0,
  '1439547067337256967': 1.0,
  '1597274845381029888': 1.0,
  '1586021183958704128': 1.0,
  '1356926480605982728': 0.0,
  '1595357036925026306': 1.0,
  '1585766233491886081': 1.0,
  '1595871258985615361': 1.0,
  '1352635736537882629': 1.0,
  '1583477966373543936': 1.0,
  '1564926450096013313': 1.0,
  '1585634359612420101': 1.0,
  '1384499047390658560': 0.0,
  '1596583748669419521': 1.0,
  '1391681495622995971': 0.0,
  '1365710259549966339': 1.0,
  '1590673118397624323': 1.0,
  '138995194334331699